## Movie Recommendation Model using Alternating Least Squares (ALS) ML algorithm

#### Requirements to run this notebook:
- Spark environment / Spark DataFrame API
- MovieLens dataset

In [0]:
%pip install findspark

Python interpreter will be restarted.
Collecting findspark
 Downloading findspark-1.4.2-py2.py3-none-any.whl (4.2 kB)
Installing collected packages: findspark
Successfully installed findspark-1.4.2
WARNING: You are using pip version 20.2.4; however, version 21.3.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-0873d776-1e19-4957-8a70-603a08c67a45/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
## check Spark environment

import findspark
findspark.init()
findspark.find()

Out[1]: '/databricks/spark'

In [0]:
## Databricks runtime and Scala version

spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion")
# Databricks Runtime Version: 8.3 (includes Apache Spark 3.1.1, Scala 2.12)

Out[2]: '8.3.x-scala2.12'

In [0]:
## SparkSession: pre-configured for Databricks Notebook by default (on Databricks Community Edition)

#from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("Databricks Shell").getOrCreate()
spark

Out[3]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

#### Process Overview
- Step 1: Load MovieLens dataset and perform exploratory data analysis
- Step 2: Split movie ratings data into training and testing datasets
- Step 3: Build ALS machine learning model to predict users movie ratings
- Step 4: Predict movie ratings for all users using the ALS model 
- Step 5: Check performance metrics of the model
- Step 6: Generate movie recommendations for existing users
- Step 7: Generate movie recommendations for a new user

### Step 1: Load MovieLens dataset and perform exploratory data analysis

In [0]:
movies = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load("/FileStore/tables/ml_latest_small_movies.csv")

movies.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [0]:
ratings = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load("/FileStore/tables/ml_latest_small_ratings.csv")

ratings.show(5)

+------+-------+------+----------+
userId|movieId|rating| timestamp|
+------+-------+------+----------+
 1| 31| 2.5|1260759144|
 1| 1029| 3.0|1260759179|
 1| 1061| 3.0|1260759182|
 1| 1129| 2.0|1260759185|
 1| 1172| 4.0|1260759205|
+------+-------+------+----------+
only showing top 5 rows

In [0]:
ratings = ratings.drop('timestamp')
ratings.show(5)

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 31| 2.5|
 1| 1029| 3.0|
 1| 1061| 3.0|
 1| 1129| 2.0|
 1| 1172| 4.0|
+------+-------+------+
only showing top 5 rows

In [0]:
ratings.printSchema()

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)

In [0]:
ratings.describe().show()

+-------+------------------+------------------+------------------+
summary| userId| movieId| rating|
+-------+------------------+------------------+------------------+
 count| 100004| 100004| 100004|
 mean| 347.0113095476181|12548.664363425463| 3.543608255669773|
 stddev|195.16383797819535|26369.198968815268|1.0580641091070326|
 min| 1| 1| 0.5|
 max| 671| 163949| 5.0|
+-------+------------------+------------------+------------------+

### Step 2: Split movie ratings data into training and testing datasets

In [0]:
train_df, test_df = ratings.randomSplit([0.8,0.2], seed = 1234)

### Step 3: Build ALS machine learning model to predict users movie ratings

In [0]:
# Load packages and modules

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:
# Create ALS model

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative=True)

In [0]:
# Create a pipleline

pipeline = Pipeline(stages=[als])

In [0]:
# Create a param_grid of hyperparameters for tuning the model

param_grid = ParamGridBuilder()\
                    .addGrid(als.rank, [5, 25, 50])\
                    .addGrid(als.maxIter, [20])\
                    .addGrid(als.regParam, [0.01, 0.1])\
                    .build()

In [0]:
# Set model evaluation metirc (evaluator) as RMSE

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [0]:
# Build the model with cross validation setup

model_cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid,
                          evaluator=evaluator, numFolds=3)

In [0]:
# Fit the model with train_df

model = model_cv.fit(train_df)

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
# Check best model hyperparameters

bestModel = model.bestModel.stages[0]

bestModel_rank = bestModel.rank
bestModel_MaxIter = bestModel._java_obj.parent().getMaxIter()
bestModel_RegParam = bestModel._java_obj.parent().getRegParam()

print("Best Model Hyperparameters:")
print(f" rank = {bestModel_rank}")
print(f" MaxIter = {bestModel_MaxIter}")
print(f" RegParam = {bestModel_RegParam}")

Best Model Hyperparameters:
 rank = 50
 MaxIter = 20
 RegParam = 0.1

### Step 4: Predict movie ratings for all users using the ALS model

In [0]:
# Generate predictions for all user ratings using the best ALS model

pred_df = bestModel.transform(test_df)

pred_df.show(5)

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 148| 1580| 4.0| 3.9542978|
 243| 1580| 3.0| 3.4397376|
 243| 1645| 3.0| 3.198189|
 243| 2122| 2.0| 2.2513754|
 516| 1088| 3.0| 3.6412926|
+------+-------+------+----------+
only showing top 5 rows

In [0]:
# Sort pred_df by "userId" and "rating" in descending order

pred_df.sort("userId","rating", ascending=False).limit(5).show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 671| 2291| 5.0| 4.0077057|
 671| 1196| 5.0| 4.430912|
 671| 2804| 5.0| 4.046247|
 671| 356| 5.0| 3.9390647|
 671| 4963| 4.5| 3.9226642|
+------+-------+------+----------+

### Step 5: Check performance metrics of the model

In [0]:
# Check RMSE evaluation metric for the model predictions

rmse = evaluator.evaluate(pred_df)
print(f"RMSE = {rmse:.4f}")

RMSE = 0.9020

#### Re-run Step 3: Build ALS machine learning model to predict users movie ratings (using updated sets of hyperparameters)

In [0]:
# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative=True)

# Create a pipleline
pipeline = Pipeline(stages=[als])

# Create a param_grid of hyperparameters for tuning the model
param_grid = ParamGridBuilder()\
                    .addGrid(als.rank, [40, 50, 60])\
                    .addGrid(als.maxIter, [40])\
                    .addGrid(als.regParam, [0.1])\
                    .build()

# Set model evaluation metirc (evaluator) as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Build the model with cross validation setup
model_cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid,
                          evaluator=evaluator, numFolds=3)

# Fit the model with train_df
model = model_cv.fit(train_df)

In [0]:
# Check best model hyperparameters

bestModel = model.bestModel.stages[0]

bestModel_rank = bestModel.rank
bestModel_MaxIter = bestModel._java_obj.parent().getMaxIter()
bestModel_RegParam = bestModel._java_obj.parent().getRegParam()

print("Best Model Hyperparameters:")
print(f" rank = {bestModel_rank}")
print(f" MaxIter = {bestModel_MaxIter}")
print(f" RegParam = {bestModel_RegParam}")

Best Model Hyperparameters:
 rank = 60
 MaxIter = 40
 RegParam = 0.1

#### Re-run Step4: Predict movie ratings for all users using the ALS model (using the updated Best Model Hyperparameters)

In [0]:
# Generate predictions for all user ratings using the best ALS model

pred_df = bestModel.transform(test_df)

pred_df.show(5)

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 148| 1580| 4.0| 3.8960123|
 243| 1580| 3.0| 3.4362233|
 243| 1645| 3.0| 3.1771646|
 243| 2122| 2.0| 2.194261|
 516| 1088| 3.0| 3.626713|
+------+-------+------+----------+
only showing top 5 rows

In [0]:
# Sort pred_df by "userId" and "rating" in descending order

pred_df.sort("userId","rating", ascending=False).limit(5).show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 671| 356| 5.0| 3.9422102|
 671| 2291| 5.0| 3.9356384|
 671| 1196| 5.0| 4.3879743|
 671| 2804| 5.0| 4.1101136|
 671| 4973| 4.5| 4.2543526|
+------+-------+------+----------+

#### Re-run Step 5: Check performance metrics of the model (using the updated Best Model Hyperparameters)

In [0]:
# Check RMSE evaluation metric for the model predictions

rmse = evaluator.evaluate(pred_df)
print(f"RMSE = {rmse:.4f}")

RMSE = 0.9022

### Step 6: Generate movie recommendations for existing users

In [0]:
# Generate top 10 movie recommendations for all users

recommendations_all_10 = bestModel.recommendForAllUsers(10)
recommendations_all_10.show(5)

+------+--------------------+
userId| recommendations|
+------+--------------------+
 1|[{123, 3.5390694}...|
 3|[{83359, 4.364339...|
 5|[{106471, 4.70427...|
 6|[{83411, 4.590722...|
 12|[{3879, 4.913202}...|
+------+--------------------+
only showing top 5 rows

In [0]:
# Check top 10 movie recommendations for userID = 1

recommendations_all_10.filter(recommendations_all_10.userId == 1).show()

+------+--------------------+
userId| recommendations|
+------+--------------------+
 1|[{123, 3.5390694}...|
+------+--------------------+

In [0]:
# Unpack the list of top 10 movieIds from recommendations column using <explode> function

recommendations_user1 = recommendations_all_10.filter(recommendations_all_10.userId == 1)

recommendations_user1.registerTempTable("ALS_recommendations_user1") 
 
recommendations_user1_exploded = spark.sql("SELECT userId, recommendation.movieId AS movieId,\
                               recommendation.rating AS pred_rating \
                               FROM ALS_recommendations_user1 \
                               LATERAL VIEW explode(recommendations) exploded_table \
                               AS recommendation")

recommendations_user1_exploded.show() 

+------+-------+-----------+
userId|movieId|pred_rating|
+------+-------+-----------+
 1| 123| 3.5390694|
 1| 858| 3.4909818|
 1| 2959| 3.468454|
 1| 232| 3.4295225|
 1| 1131| 3.379272|
 1| 5269| 3.3775406|
 1| 750| 3.3734868|
 1| 26587| 3.365525|
 1| 50| 3.3597205|
 1| 1859| 3.3580768|
+------+-------+-----------+

In [0]:
# Add movie info (movie title and genres) to top 10 movie recommendations data (with movieIds and ratings)

recommendations_user1_exploded.join(movies, ["movieId"], "left").show()

+-------+------+-----------+--------------------+--------------------+
movieId|userId|pred_rating| title| genres|
+-------+------+-----------+--------------------+--------------------+
 123| 1| 3.5390694|Chungking Express...|Drama|Mystery|Rom...|
 858| 1| 3.4909818|Godfather, The (1...| Crime|Drama|
 2959| 1| 3.468454| Fight Club (1999)|Action|Crime|Dram...|
 232| 1| 3.4295225|Eat Drink Man Wom...|Comedy|Drama|Romance|
 1131| 1| 3.379272|Jean de Florette ...| Drama|Mystery|
 5269| 1| 3.3775406|Piano Teacher, Th...| Drama|
 750| 1| 3.3734868|Dr. Strangelove o...| Comedy|War|
 26587| 1| 3.365525|Decalogue, The (D...| Crime|Drama|Romance|
 50| 1| 3.3597205|Usual Suspects, T...|Crime|Mystery|Thr...|
 1859| 1| 3.3580768|Taste of Cherry (...| Drama|
+-------+------+-----------+--------------------+--------------------+

### Step 7: Generate movie recommendations for a new user

#### Step 7.1: Add new user data to the existing ratings data

In [0]:
# Let's assume a new user's favorite movies are the following:
# Toy Story (1995), Lion King, The (1994), Shrek (2001), Finding Nemo (2003) 
# which can be converted as a list of movieIds as below:
new_user_movieIds = [1, 364, 4306, 6377]

# set new userId as the highest userId value among the existing users + 1
new_userId = ratings.agg({"userId": "max"}).first()[0] + 1
rating_max = 5.0

# Create a list of new user's movie rating data
new_user_ratings = [(new_userId, movieId, rating_max) for movieId in new_user_movieIds]

# ratings.columns = ['userId', 'movieId', 'rating']

# Create a new dataframe that contains new user's ratings
new_user_df = spark.createDataFrame(new_user_ratings, ratings.columns)
new_user_df.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 672| 1| 5.0|
 672| 364| 5.0|
 672| 4306| 5.0|
 672| 6377| 5.0|
+------+-------+------+

In [0]:
# Add new_user_df to the existing ratings data

ratings_all_new = ratings.union(new_user_df)

ratings_all_new.filter(ratings_all_new.userId == new_userId).show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 672| 1| 5.0|
 672| 364| 5.0|
 672| 4306| 5.0|
 672| 6377| 5.0|
+------+-------+------+

#### Step 7.2: Split movie ratings data into training and testing datasets

In [0]:
train_df, test_df = ratings_all_new.randomSplit([0.8,0.2], seed = 1234)

#### Step 7.3: Build ALS machine learning model to predict users movie ratings

In [0]:
## Rebuild ALS model with updated ratings datasets that include new user's ratings

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative=True)

pipeline = Pipeline(stages=[als])

# param_grid contains only the bestModel hyperparameters that were obtained above
param_grid = ParamGridBuilder()\
                    .addGrid(als.rank, [bestModel_rank])\
                    .addGrid(als.maxIter, [bestModel_MaxIter])\
                    .addGrid(als.regParam, [bestModel_RegParam])\
                    .build()

# Set model evaluation metirc (evaluator) as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Build the model with cross validation setup
model_cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid,
                          evaluator=evaluator, numFolds=3)

# Fit the model with train_df
model = model_cv.fit(train_df)

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

#### Step 7.4: Predict movie ratings for all users using the ALS model

In [0]:
# Generate predictions for all user ratings using the new ALS model

pred_df = model.transform(test_df)

pred_df.show(5)

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 463| 1959| 4.0| 3.630198|
 463| 3175| 3.0| 3.2008274|
 471| 1580| 3.0| 3.68009|
 623| 1580| 3.5| 3.8037965|
 85| 471| 3.0| 3.2861629|
+------+-------+------+----------+
only showing top 5 rows

In [0]:
# Sort pred_df by "userId" and "rating" in descending order

pred_df.sort("userId","rating", ascending=False).limit(5).show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 672| 1| 5.0| 4.7037663|
 671| 2804| 5.0| 3.9212635|
 671| 3114| 5.0| 3.9815636|
 671| 1035| 5.0| 4.036878|
 671| 2291| 5.0| 3.7752411|
+------+-------+------+----------+

In [0]:
# Check best model hyperparameters, which should remain the same as given above

bestModel = model.bestModel.stages[0]

bestModel_rank = bestModel.rank
bestModel_MaxIter = bestModel._java_obj.parent().getMaxIter()
bestModel_RegParam = bestModel._java_obj.parent().getRegParam()

print("Best Model Hyperparameters:")
print(f" rank = {bestModel_rank}")
print(f" MaxIter = {bestModel_MaxIter}")
print(f" RegParam = {bestModel_RegParam}")

Best Model Hyperparameters:
 rank = 60
 MaxIter = 40
 RegParam = 0.1

#### Step 7.5: Check performance metrics of the model

In [0]:
# Check RMSE evaluation metric for the model predictions

rmse = evaluator.evaluate(pred_df)
print(f"RMSE = {rmse:.4f}")

RMSE = 0.9015

#### Step 7.6: Generate movie recommendations for existing users

In [0]:
# Generate top 10 movie recommendations for all users

recommendations_all_10 = bestModel.recommendForAllUsers(10)
recommendations_all_10.show(5)

+------+--------------------+
userId| recommendations|
+------+--------------------+
 1|[{2563, 3.572302}...|
 3|[{59684, 4.423637...|
 5|[{59684, 4.63246}...|
 6|[{67504, 5.007091...|
 12|[{3879, 4.8949223...|
+------+--------------------+
only showing top 5 rows

#### Step 7.7: Retrieve movie recommendations for the new user

In [0]:
# Unpack the list of top 10 movieIds from recommendations column using <explode> function

recommendations_new_user = recommendations_all_10.filter(recommendations_all_10.userId == new_userId)

recommendations_new_user.registerTempTable("ALS_recommendations_new_user") 
 
recommendations_new_user_exploded = spark.sql("SELECT userId, recommendation.movieId AS movieId,\
                               recommendation.rating AS pred_rating \
                               FROM ALS_recommendations_new_user \
                               LATERAL VIEW explode(recommendations) exploded_table \
                               AS recommendation")

recommendations_new_user_exploded.show() 

+------+-------+-----------+
userId|movieId|pred_rating|
+------+-------+-----------+
 672| 67504| 5.357431|
 672| 83411| 5.357431|
 672| 83318| 5.357431|
 672| 527| 5.1312346|
 672| 59684| 5.086157|
 672| 54328| 5.0745444|
 672| 31435| 5.051767|
 672| 1148| 4.985118|
 672| 4306| 4.983896|
 672| 905| 4.95656|
+------+-------+-----------+

In [0]:
# Add movie info (movie title and genres) to top 10 movie recommendations data (with movieIds and ratings)

recommendations_new_user_exploded.join(movies, ["movieId"], "left").toPandas()

Out[42]:

,movieId,userId,pred_rating,title,genres
0,67504,672,5.357431,Land of Silence and Darkness (Land des Schweig...,Documentary
1,83411,672,5.357431,Cops (1922),Comedy
2,83318,672,5.357431,"Goat, The (1921)",Comedy
3,527,672,5.131235,Schindler's List (1993),Drama|War
4,59684,672,5.086157,Lake of Fire (2006),Documentary
5,54328,672,5.074544,My Best Friend (Mon meilleur ami) (2006),Comedy
6,31435,672,5.051767,Rory O'Shea Was Here (Inside I'm Dancing) (2004),Drama
7,1148,672,4.985118,Wallace & Gromit: The Wrong Trousers (1993),Animation|Children|Comedy|Crime
8,4306,672,4.983896,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Ro...
9,905,672,4.956560,It Happened One Night (1934),Comedy|Romance


In [0]:
Top10_movies = recommendations_new_user_exploded.join(movies, ["movieId"], "left").select("title").toPandas()

In [0]:
Top10_movies

Out[44]:

,title
0,Land of Silence and Darkness (Land des Schweig...
1,Cops (1922)
2,"Goat, The (1921)"
3,Schindler's List (1993)
4,Lake of Fire (2006)
5,My Best Friend (Mon meilleur ami) (2006)
6,Rory O'Shea Was Here (Inside I'm Dancing) (2004)
7,Wallace & Gromit: The Wrong Trousers (1993)
8,Shrek (2001)
9,It Happened One Night (1934)
